In [ ]:

df_C_pred=pd.DataFrame()
for s in scaled_df.columns:
    
    # prepare data:
    # Separate size of data   
    train_size=int(len(scaled_df) * 0.9) 
    df_sector= scaled_df[[s]]
    df_sector.index = pd.to_datetime(df_sector.index, format='%Y')
    # Train and Validate
    df_tr=df_sector.iloc[:train_size]
    df_val=df_sector.iloc[train_size:]
       
    
    # Build model ARIMA with order=(1,1,1) or (2,1,1)
    ## We can select model here for each sector by including the condition....
    
    # Build model ARIMA with oder chose manually
    if s=='TRANSPORT':
        model_val=ARIMA(df_tr, order=(2,2,3))# or maybe (4,2,3)
        model = ARIMA(df_sector, order=(2,2,3))# or maybe (4,2,3)
    if s=='INDUSTRIAL':
        model_val = ARIMA(df_tr, order=(2,2,3))
        model = ARIMA(df_sector, order=(2,2,3))
    if s=='RESIDENTIAL':
        model_val = ARIMA(df_tr, order=(4,2,2)) # or maybe (3,1,2)
        model = ARIMA(df_sector, order=(4,2,2)) # or maybe (3,1,2)
    if s=='COMMERCIAL, SERVICES, PUBLIC':
        model_val = ARIMA(df_tr, order=(1,1,1))
        model = ARIMA(df_sector, order=(1,1,1))       
    if s=='AGRICULTURE, FISHING AND MINING':
        model_val = ARIMA(df_tr, order=(4,1,3))
        model = ARIMA(df_sector, order=(4,1,3))
    if s=='CONSTRUCTION AND OTHERS':
        model_val = ARIMA(df_tr, order=(3,0,3))
        model = ARIMA(df_sector, order=(3,0,3))
    if s=='NON-ENERGY CONSUMPTION':
        model_val = ARIMA(df_tr, order=(0,2,1))
        model = ARIMA(df_sector, order=(0,2,1))
    if s=='FINAL CONSUMPTION':
        model_val = ARIMA(df_tr, order=(2,2,2))
        model = ARIMA(df_sector, order=(2,2,2))
    
    model_val_fit=model_val.fit()
    model_fit = model.fit()
    
    
    # Validation:
    
    forecast_val = model_val_fit.predict(start=df_val.index[0], end=df_val.index[-1])

    # Create a Pandas DataFrame with the forecasted values and set the index
    df_forecast_val = pd.DataFrame(data=forecast_val, index=df_val.index)

    mape_val=MAPE(df_val, df_forecast_val)
    # Plot

    #plt.plot(df_sector, "b", label= "Actual")
    plt.plot(df_forecast_val, "r", label="Forecast")
    

    
    #Predict:
    
    # Specify the number of future steps to forecast
    periods = 20  # Adjust as needed
    
    # Specify the start date
    start_date = pd.to_datetime('2022-01-01')

    # Prediction for future time points
    data_pred = model_fit.predict(start=start_date, end=start_date+ pd.DateOffset(years=periods))

    # Create a date range for the forecasted period
    forecast_index = pd.date_range(start=start_date, periods=periods+1, freq='A')

    # Create a Pandas DataFrame with the forecasted values and set the index
    df_forecast = pd.DataFrame(data=list(data_pred), index=forecast_index, columns=[s])

    # Plot
    plt.plot(df_sector, "b", label="Actual")
    plt.plot(df_forecast, "r", label="Predict")

    #plt.title(f"{s}")
    plt.title(f'{s}-MAPE={mape_val}')
    plt.legend()
    plt.show()
    
    #Concatenate df
    df_C_pred=pd.concat([df_C_pred,df_forecast],axis=1)
# Round the values of each column in the dataframe with 2 decimals
df_C_pred = df_C_pred.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)  
df_C_pred.index=df_C_pred.index.year
# Save the df_forecast to excel
file_name='df_C_pred.xlsx'
df_C_pred.to_excel(file_name)